### This CNN uses two conv-conv-pool blocks with 16,16 and 32,32 filters in the convolution layers. Two final dense layers.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, InputLayer
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
import os
root_logdir = os.path.join(os.curdir, "conv_conv_pool_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [3]:
batch_size = 32
target_size = (175,175)

train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1,
                                  validation_split=0.1)

test_datagen = ImageDataGenerator(rescale=1./255, 
                                  validation_split=0.1)


train_generator = train_datagen.flow_from_directory('../data/images/train',
                                                    color_mode='rgb',
                                                    target_size=target_size,
                                                    batch_size=batch_size,
                                                    class_mode="categorical",
                                                    subset='training')

validation_generator = test_datagen.flow_from_directory('../data/images/train',
                                                       color_mode='rgb',
                                                       target_size=target_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical',
                                                       subset='validation')

holdout_generator = test_datagen.flow_from_directory('../data/images/holdout',
                                                       color_mode='rgb',
                                                       target_size=target_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')

labels = (train_generator.class_indices)
labels = dict((v, k) for k,v in labels.items())
labels


Found 2051 images belonging to 6 classes.
Found 225 images belonging to 6 classes.
Found 251 images belonging to 6 classes.


{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

In [9]:
model=Sequential()

model.add(Conv2D(16, (3,3), padding='same', input_shape=(target_size[0], target_size[1], 3), activation='relu'))
model.add(Conv2D(16, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))


model.compile(loss='categorical_crossentropy',
             optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 175, 175, 16)      2368      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 175, 175, 16)      12560     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 87, 87, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 87, 87, 32)        4640      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 87, 87, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 43, 43, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 59168)            

In [7]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("conv_conv_pool.h5")
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir())


history = model.fit(
    train_generator,
    validation_data = validation_generator, 
    verbose=1,
    epochs = 50,
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

Epoch 1/50
65/65 [==============================] - 80s 1s/step - loss: 1.7032 - accuracy: 0.2565 - val_loss: 1.3917 - val_accuracy: 0.4000
Epoch 2/50
65/65 [==============================] - 74s 1s/step - loss: 1.4271 - accuracy: 0.4180 - val_loss: 1.3038 - val_accuracy: 0.4533
Epoch 3/50
65/65 [==============================] - 74s 1s/step - loss: 1.2789 - accuracy: 0.4950 - val_loss: 1.2519 - val_accuracy: 0.4933
Epoch 4/50
65/65 [==============================] - 78s 1s/step - loss: 1.2307 - accuracy: 0.5211 - val_loss: 1.1692 - val_accuracy: 0.5689
Epoch 5/50
65/65 [==============================] - 73s 1s/step - loss: 1.1994 - accuracy: 0.5425 - val_loss: 1.1588 - val_accuracy: 0.4889
Epoch 6/50
65/65 [==============================] - 74s 1s/step - loss: 1.0450 - accuracy: 0.6210 - val_loss: 1.0801 - val_accuracy: 0.5556
Epoch 7/50
65/65 [==============================] - 74s 1s/step - loss: 1.0506 - accuracy: 0.6073 - val_loss: 1.1167 - val_accuracy: 0.5289
Epoch 8/50
65/65 [==

In [8]:
score = model.evaluate(holdout_generator, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.788844645023346


In [10]:
max(history.history['val_accuracy'])

0.746666669845581